<a href="https://colab.research.google.com/github/goyalvansh4/Mern-Stack/blob/master/quickstarts/Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required libraries
from fastapi import FastAPI
from TTS.api import TTS
from pyngrok import ngrok
import torch
import os

# Initialize FastAPI app
app = FastAPI()

# Load the model with GPU if available
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=torch.cuda.is_available())

# Supported languages
supported_languages = [
    "en", "es", "fr", "de", "it", "pt", "pl", "tr", "ru",
    "nl", "cs", "ar", "zh-cn", "hu", "ko", "ja", "hi"
]

# Speaker audio files (Ensure these are uploaded to Google Colab under 'speakers/' folder)
speaker_wav_files = {
    "en": "speakers/en.wav",
    "hi": "speakers/hi.wav",
    "es": "speakers/es.wav",
    "fr": "speakers/fr.wav",
    "de": "speakers/de.wav",
    "it": "speakers/it.wav"
    # Add more if available
}

# API Route: Generate Voice
@app.get("/generate-voice/")
async def generate_voice(text: str, language: str = "en", speaker: str = None):
    if language not in supported_languages:
        return {"error": f"Language '{language}' is not supported. Choose from {supported_languages}"}

    output_path = f"generated_audio/{language}.wav"
    os.makedirs("generated_audio", exist_ok=True)

    try:
        speaker_wav = speaker_wav_files.get(language) if speaker is None else speaker

        if not os.path.exists(speaker_wav):
            return {"error": f"Speaker WAV file not found: {speaker_wav}"}

        tts.tts_to_file(
            text=text,
            language=language,
            speaker_wav=speaker_wav,
            file_path=output_path,
            speed=1.2  # Adjust speed if needed
        )
        return {"message": "Generated Voice File", "file_path": output_path}
    except Exception as e:
        return {"error": str(e)}

# Start FastAPI server with ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run Uvicorn
import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)